In [1]:
# Hide all code cells (including this one) and add a toggle button
from IPython.display import HTML, display

display(HTML('''
<script>
  function toggleCodeCells() {
    var codeCells = document.querySelectorAll('div.input');
    var anyVisible = Array.from(codeCells).some(c => c.style.display !== 'none');
    codeCells.forEach(c => c.style.display = anyVisible ? 'none' : '');
  }

  // Add toggle button
  const button = document.createElement('button');
  button.textContent = 'Show/Hide Code';
  button.style.position = 'fixed';
  button.style.top = '10px';
  button.style.right = '10px';
  button.style.zIndex = 1000;
  button.style.padding = '6px 10px';
  button.style.background = '#1976D2';
  button.style.color = 'white';
  button.style.border = 'none';
  button.style.borderRadius = '6px';
  button.onclick = toggleCodeCells;
  document.body.appendChild(button);

  // Hide all code cells (including this one) on load
  document.addEventListener('DOMContentLoaded', function() {
    var firstCell = document.querySelector('div.input');
    if (firstCell) firstCell.style.display = 'none';
    toggleCodeCells();
  });
</script>
'''))



# Maxwell Model — Viscoelastic Fluid (Series)

Many real materials — such as polymers, biological tissues, and asphalt — exhibit both *elastic* and *viscous* behavior.  
The **Maxwell model** represents a spring and a dashpot in **series** and captures stress relaxation and unbounded creep.

$$
\sigma + \tau \dot{\sigma} = \eta \dot{\gamma}, \qquad \tau = \frac{\eta}{E}
$$

Where:  
- $ \sigma $ — shear stress  
- $ \dot{\gamma} $ — shear rate  
- $ E $ — elastic modulus (spring stiffness)  
- $ \eta $ — viscosity (dashpot resistance)  
- $ \tau = \eta / E $ — relaxation time


In [2]:

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox

def stress_relaxation_maxwell(t, E, eta, gamma0):
    tau = eta / E
    return E * gamma0 * np.exp(-t / tau)

def creep_maxwell(t, E, eta, sigma0):
    return sigma0 / E + (sigma0 / eta) * t

# Sliders (two-column layout)
E_slider = widgets.FloatLogSlider(value=1e5, base=10, min=3, max=7, step=0.1, description='E [Pa]')
eta_slider = widgets.FloatLogSlider(value=1e6, base=10, min=4, max=8, step=0.1, description='η [Pa·s]')
gamma0_slider = widgets.FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='γ₀')
sigma0_slider = widgets.FloatSlider(value=100, min=10, max=1000, step=10, description='σ₀ [Pa]')
T_slider = widgets.FloatLogSlider(value=10, base=10, min=-1, max=3, step=0.1, description='T_process [s]')
tmax_slider = widgets.FloatLogSlider(value=10, base=10, min=-2, max=4, step=0.1, description='t_max [s]')

ui_left = VBox([E_slider, eta_slider, gamma0_slider])
ui_right = VBox([sigma0_slider, T_slider, tmax_slider])
ui = HBox([ui_left, ui_right])

def plot_maxwell(E, eta, gamma0, sigma0, T_process, tmax):
    tau = eta / E
    De = tau / T_process
    t = np.linspace(0, tmax, 400)

    sigma = stress_relaxation_maxwell(t, E, eta, gamma0)
    gamma = creep_maxwell(t, E, eta, sigma0)

    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    ax[0].plot(t, sigma, lw=2, label='Maxwell')
    ax[0].set_title('Stress Relaxation')
    ax[0].set_xlabel('Time [s]')
    ax[0].set_ylabel('Stress σ [Pa]')
    ax[0].grid(True)
    ax[0].legend()

    ax[1].plot(t, gamma, lw=2, label='Maxwell')
    ax[1].set_title('Creep')
    ax[1].set_xlabel('Time [s]')
    ax[1].set_ylabel('Strain γ [-]')
    ax[1].grid(True)
    ax[1].legend()

    plt.tight_layout()
    plt.show()

    print(f"Relaxation time τ = {tau:.3g} s")
    print(f"Deborah number De = {De:.3g}")

out = widgets.interactive_output(
    plot_maxwell,
    {'E': E_slider, 'eta': eta_slider, 'gamma0': gamma0_slider,
     'sigma0': sigma0_slider, 'T_process': T_slider, 'tmax': tmax_slider}
)

display(ui, out)


Output()


---

## 👩‍🏫 Teacher Information (Maxwell)

**Purpose.** Explore stress relaxation and creep with the Maxwell model (series spring–dashpot).  
**Key relation.** $ \tau = \eta / E $. Large $ \tau $ → more elastic memory; small $ \tau $ → more fluid-like.

**Use in class.**
1. Open in Colab via badge → *Run all*.
2. Vary $E$, $\eta$, $\gamma_0$, and $\sigma_0$; observe:
   - Stress relaxation: exponential decay of $\sigma(t)$.
   - Creep: linear growth of $\gamma(t)$.
3. Discuss the role of $T_\text{process}$ and Deborah number $De=\tau/T_\text{process}$.

**Notes.** Each student runs an isolated Colab session; no shared state.
